## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/93-googleq-a-train-glove-sgdr/model.h5'
tokenizer_path = '/kaggle/input/93-googleq-a-train-glove-sgdr/tokenizer.joblib'
category_encoder_path = '/kaggle/input/93-googleq-a-train-glove-sgdr/category_encoder.joblib'
host_encoder_path = '/kaggle/input/93-googleq-a-train-glove-sgdr/host_encoder.joblib'
scaler_path = '/kaggle/input/93-googleq-a-train-glove-sgdr/scaler.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

    
# Text length
test['question_title_len'] = test['question_title'].apply(lambda x : len(x))
test['question_body_len'] = test['question_body'].apply(lambda x : len(x))
test['answer_len'] = test['answer'].apply(lambda x : len(x))

# Word count
test['question_title_word_cnt'] = test['question_title'].apply(lambda x : len(x.split(' ')))
test['question_body_word_cnt'] = test['question_body'].apply(lambda x : len(x.split(' ')))
test['answer_word_cnt'] = test['answer'].apply(lambda x : len(x.split(' ')))

# Unique word count
test['question_title_unique_word_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' '))))
test['question_body_unique_word_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' '))))
test['answer_unique_word_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' '))))

# Misspelings count
misspelling_mapping_set = set(misspelling_mapping.keys())
test['question_title_misspelings_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['question_body_misspelings_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['answer_misspelings_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))

# Meta features
test['question_title_count'] = test['question_title'].apply(lambda x : x.count('?'))
test['question_body_count'] = test['question_body'].apply(lambda x : x.count('?'))
test['title_exp_count'] = test['question_title'].apply(lambda x : x.count('why'))
test['body_exp_count'] = test['question_body'].apply(lambda x : x.count('why'))
test['title_choice_count'] = test['question_title'].apply(lambda x : x.count('or'))
test['body_choice_count'] = test['question_body'].apply(lambda x : x.count('or'))
test['question_aswer_ratio'] = test['question_title_word_cnt'] / test['answer_word_cnt']
test['same_author'] = test['question_user_page'] == test['answer_user_page']

for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 100000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 100
ANSWER_MAX_LEN = 100

## Test set

In [6]:
tokenizer = load(tokenizer_path)
ohe_category = load(category_encoder_path)
ohe_host = load(host_encoder_path)
scaler = load(scaler_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']
X_test_category = ohe_category.transform(test[['category']]).toarray()
X_test_host = ohe_host.transform(test[['host']]).toarray()
X_test_dense = test[['question_title_len', 'question_title_word_cnt', 'question_title_unique_word_cnt', 'question_title_misspelings_cnt', 
                     'question_body_len', 'question_body_word_cnt', 'question_body_unique_word_cnt', 'question_body_misspelings_cnt', 
                     'answer_len', 'answer_word_cnt', 'answer_unique_word_cnt', 'answer_misspelings_cnt', 
                     'question_title_count', 'question_body_count', 'title_exp_count', 'body_exp_count', 'title_choice_count', 
                     'body_choice_count', 'body_choice_count', 'question_aswer_ratio', 'same_author']]

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test_dense = scaler.transform(X_test_dense)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq, X_test_category, X_test_host, X_test_dense]

# Model

In [7]:
# use_embed = hub.load(module_url)

# def USEEmbedding(x):
#     return use_embed(tf.squeeze(tf.cast(x, tf.string)))

# def encoder_block(input_layer):
#     encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
#     return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
embedding_name = 'GloVe'
print('n unknown words "%s": %d' % (embedding_name, len(glove_unknown_words)))
print('"%s" embedding shape: %s' % (embedding_name, glove_embedding_matrix.shape))

n unknown words "GloVe": 19913
"GloVe" embedding shape: (58007, 300)


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.3)(title)
title = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(title)
title = Dropout(0.5)(title)
title = Dense(512, kernel_initializer='glorot_uniform')(title)
title = BatchNormalization()(title)
title_out = Activation('relu')(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = SpatialDropout1D(0.3)(body)
body = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(body)
body = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(body)
body = Dropout(0.5)(body)
body = Dense(512, kernel_initializer='glorot_uniform')(body)
body = BatchNormalization()(body)
body_out = Activation('relu')(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = SpatialDropout1D(0.3)(answer)
answer = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(answer)
answer = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(answer)
answer = Dropout(0.5)(answer)
answer = Dense(512, kernel_initializer='glorot_uniform')(answer)
answer = BatchNormalization()(answer)
answer_out = Activation('relu')(answer)

# Category
input_category = Input(shape=(X_test_category.shape[1],), dtype=tf.float32, name='input_category')
category = Embedding(X_test_category.shape[1], X_test_category.shape[1]//2)(input_category)
category_out = GlobalAveragePooling1D()(category)

# Host
input_host = Input(shape=(X_test_host.shape[1],), dtype=tf.float32, name='input_host')
host = Embedding(X_test_host.shape[1], X_test_host.shape[1]//2)(input_host)
host_out = GlobalAveragePooling1D()(host)

# Dense features
input_dense = Input(shape=(X_test_dense.shape[1],), dtype=tf.float32, name='input_dense')

# Output
output = Concatenate()([title_out, body_out, answer_out, category_out, host_out, input_dense])
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='glorot_uniform')(output)

model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq, input_category, input_host, input_dense], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_body_seq (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     17402100    input_body_seq[0][0]             
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.910633,0.620891,0.110502,0.693173,0.783052,0.673040,0.632902,0.604352,0.329458,...,0.878153,0.865586,0.585369,0.942487,0.934582,0.768365,0.080353,0.068122,0.549986,0.896865
1,46,0.830734,0.518245,0.004433,0.678414,0.755574,0.887390,0.528793,0.442899,0.106564,...,0.704999,0.918298,0.633488,0.937792,0.958833,0.837930,0.884828,0.156433,0.152359,0.850037
2,70,0.912799,0.706798,0.069475,0.736241,0.895679,0.815414,0.614149,0.610293,0.324849,...,0.882592,0.833948,0.570877,0.910108,0.925349,0.759738,0.048599,0.049964,0.566678,0.877672
3,132,0.842570,0.504297,0.005495,0.717812,0.782388,0.911022,0.524356,0.436755,0.151699,...,0.724876,0.941133,0.657490,0.959792,0.977933,0.879929,0.820866,0.176492,0.545662,0.879195
4,200,0.922848,0.611876,0.016701,0.789370,0.866556,0.861350,0.607714,0.526843,0.306609,...,0.842283,0.956049,0.677076,0.984952,0.985881,0.894584,0.174161,0.082142,0.743350,0.940855


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.872172,0.569317,0.041239,0.707166,0.789112,0.838515,0.574922,0.490643,0.224168,...,0.766841,0.918680,0.648946,0.954075,0.964495,0.848315,0.549950,0.142409,0.496315,0.892572
std,2812.670060,0.044670,0.126676,0.074674,0.078951,0.089905,0.115792,0.040245,0.070768,0.104242,...,0.093017,0.023631,0.030281,0.015699,0.016108,0.031922,0.310703,0.058705,0.237371,0.026054
min,39.000000,0.730772,0.341416,0.001575,0.346654,0.261533,0.217487,0.504073,0.378755,0.041873,...,0.590838,0.817795,0.530070,0.890565,0.869508,0.705889,0.008333,0.018780,0.063453,0.822369
25%,2572.000000,0.837512,0.451813,0.005281,0.675096,0.749223,0.827940,0.539262,0.436945,0.129185,...,0.679204,0.906219,0.631139,0.945659,0.957624,0.830750,0.227552,0.102487,0.287291,0.874711
50%,5093.000000,0.869875,0.557050,0.009706,0.714191,0.788449,0.882759,0.570619,0.467820,0.220123,...,0.775666,0.922942,0.650931,0.953255,0.968057,0.853602,0.667187,0.142462,0.506428,0.892783
75%,7482.000000,0.910735,0.677389,0.038295,0.753460,0.833496,0.901173,0.606203,0.529595,0.309822,...,0.855483,0.934208,0.668078,0.964369,0.975761,0.869834,0.815837,0.183494,0.696111,0.914193
max,9640.000000,0.965373,0.844219,0.479215,0.880015,0.974114,0.955503,0.676638,0.698449,0.483904,...,0.918287,0.965449,0.713387,0.988935,0.989387,0.914600,0.934152,0.345678,0.933542,0.946937
